In [1]:
!pip install tensorflow_addons

  Using cached tensorflow_addons-0.20.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (591 kB)
  Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)


In [2]:
#Imports and settings

import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import sys

import tensorflow as tf
from tqdm.notebook import tqdm
import tensorflow_addons as tfa


from sklearn.model_selection import train_test_split

np.set_printoptions(suppress=True)

/opt/conda/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/opt/conda/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're usin

In [3]:
# load the features and labels (subtract 1 as the labels aren't indexed from 0)
ytest = np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/test/y_test.txt')-1
ytrain = np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/train/y_train.txt')-1
# load the x,y,z body accelerations test data
xx_test=np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/test/Inertial Signals/body_acc_x_test.txt')
yy_test=np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/test/Inertial Signals/body_acc_y_test.txt')
zz_test=np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/test/Inertial Signals/body_acc_z_test.txt')
# concatenate the arrays along the last dimension
xtest = np.concatenate((xx_test[:,:,None],yy_test[:,:,None],zz_test[:,:,None]),axis=2)
# (using None here adds an extra dimension of size 1 to the end of the array)
# follow the same approach for the train data
xx_train=np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/train/Inertial Signals/body_acc_x_train.txt')
yy_train=np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/train/Inertial Signals/body_acc_y_train.txt')
zz_train=np.loadtxt('/home/jovyan/datasets/UCI_HAR_DATASET/train/Inertial Signals/body_acc_z_train.txt')
xtrain = np.concatenate((xx_train[:,:,None],yy_train[:,:,None],zz_train[:,:,None]),axis=2)
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(7352, 128, 3)
(2947, 128, 3)
(7352,)
(2947,)


In [4]:
#tensorboad set up
logdir = '/home/jovyan/logs/Project/Part2'

In [17]:
#Recreate the tensors
x_input, x_val, y_input, y_val = train_test_split(xtrain, ytrain, test_size=0.2, random_state=10)

#one-hot encoding for labels
y_input = tf.keras.utils.to_categorical(y_input)
y_val = tf.keras.utils.to_categorical(y_val)

# Convert your input and validation data to a tensor so that you can pass it into your classifier
x_input = tf.convert_to_tensor(x_input, tf.float32)
x_val = tf.convert_to_tensor(x_val, tf.float32)
y_input = tf.convert_to_tensor(y_input, tf.float32)
y_val = tf.convert_to_tensor(y_val, tf.float32)

In [18]:
# Keras API model WORKING

#We create the input layer and combine with further dense layers,
input_layer = tf.keras.Input(shape=(128, 3))
output_layer = tf.keras.layers.LSTM(128)(input_layer)
output_layer = tf.keras.layers.BatchNormalization()(output_layer)
output_layer = tf.keras.layers.Dense(64, activation='relu')(output_layer)
# output_layer = tf.keras.layers.GlobalAveragePooling1D()(output_layer)  #remove
output_layer = tf.keras.layers.Dropout(0.5)(output_layer)

# # Add a softmax layer with  output units:
output_layer = tf.keras.layers.Dense(6, activation='softmax')(output_layer)

model = tf.keras.Model(inputs=input_layer,outputs=output_layer)

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 128, 3)]          0         
                                                                 
 lstm (LSTM)                 (None, 128)               67584     
                                                                 
 batch_normalization_4 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 6)                 390       
                                                           

In [20]:
# Keras API model WORKING


#We create the input layer and combine with further dense layers,
input_layer = tf.keras.Input(shape=(128, 3))
rnn_layer = tf.keras.layers.RNN(tf.keras.layers.SimpleRNNCell(32))(input_layer) # 4 is the number of units
output_layer = tf.keras.layers.BatchNormalization()(rnn_layer)
# output_layer = tf.keras.layers.Dense(64, activation='relu')(output_layer)
# output_layer = tf.keras.layers.GlobalAveragePooling1D()(output_layer)  #remove

# # Add a softmax layer with  output units:
output_layer = tf.keras.layers.Dense(6, activation='softmax')(output_layer)

model = tf.keras.Model(inputs=input_layer,outputs=output_layer)

model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 128, 3)]          0         
                                                                 
 rnn_5 (RNN)                 (None, 32)                1152      
                                                                 
 batch_normalization_5 (Batc  (None, 32)               128       
 hNormalization)                                                 
                                                                 
 dense_9 (Dense)             (None, 6)                 198       
                                                                 
Total params: 1,478
Trainable params: 1,414
Non-trainable params: 64
_________________________________________________________________


In [22]:
# Compile model

# Callbacks
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
# create a callback that will stop training if the validation loss hasn't improved for 2 epochs
#early_stopping_callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)]
# initialize tqdm callback with default parameters
tqdm_callback = tfa.callbacks.TQDMProgressBar()


#The compilation and training then proceeds as per the sequential model,
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
loss='categorical_crossentropy', metrics=['accuracy'])



In [23]:
#fit model

model.fit(x_input, y_input, 
          epochs=200, 
          batch_size=128,
          validation_data=(x_val, y_val),
          callbacks=[tensorboard_callback, tqdm_callback]
         )

Training:   0%|           0/200 ETA: ?s,  ?epochs/s

Epoch 1/200


0/46           ETA: ?s - 

Epoch 1/200
46/46 [==============================] - 4s 77ms/step - loss: 1.6334 - accuracy: 0.2850 - val_loss: 1.5520 - val_accuracy: 0.2495
Epoch 2/200


0/46           ETA: ?s - 

...

Epoch 198/200
46/46 [==============================] - 3s 72ms/step - loss: 1.7267 - accuracy: 0.2736 - val_loss: 1.7692 - val_accuracy: 0.1434
Epoch 199/200


0/46           ETA: ?s - 

Epoch 199/200
46/46 [==============================] - 3s 72ms/step - loss: 1.6978 - accuracy: 0.2809 - val_loss: 1.7043 - val_accuracy: 0.2814
Epoch 200/200


0/46           ETA: ?s - 

Epoch 200/200
46/46 [==============================] - 3s 70ms/step - loss: 1.6810 - accuracy: 0.2787 - val_loss: 1.6783 - val_accuracy: 0.2848
